In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from math import sqrt
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from keras import metrics
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)

#new_data_Modena_backfill_forwfill_index
#data = pd.read_csv('/Users/alket/Desktop/dati/new_data_backfill_forwfill.csv',index_col = 0)
data = pd.read_csv('/Users/alket/Desktop/dati/new_data_Modena_backfill_forwfill.csv',index_col = 0)
             

# aggregazione dati per celle
agg_by_cell = data.groupby(by = ['cell_num'])

In [2]:
trend_data = []
residual_data = []
seasonal_data = []

In [3]:
counter = 0
for i, k in agg_by_cell:
    counter+=1
    print('-->  ', counter)
    dates4dec = []
    cell_values = []
    k = k.iloc[::4,:]
    for index, row in k.iterrows():
        
        date = row['date']
        h = str(row['hours'])
   
        h = h.split('.')
    
        if len(h[0])<2:
            h = h[1]+h[0]
        else: 
            h = h[0]
   
        minutes = str(row['minutes'])
        m = ''
        minutes = minutes.split('.')
        if len(minutes[0])<2: 
            m = minutes[0] +'0'
        else: 
            m = minutes[0]
        
        data_f = date+' '+h+':'+m+':'+'00'
        cell_values.append(row['nr_people'])
        dates4dec.append(data_f) 
        
    dict_i = {'ds': dates4dec, 'y':cell_values}
    data4deco = pd.DataFrame(dict_i, index=None, columns=None)  
    #print(data4deco)
    data4deco['ds'] = pd.to_datetime(data4deco['ds'])
    data4deco = data4deco.set_index('ds')
    #print(data4deco)
    decomp = decompose(data4deco['y'], period=96)
    #with plt.rc_context():
    #    plt.rc("figure", figsize=(20,8))
    #    decomp.plot()
    #    plt.show()
    trend = decomp.trend.values
    seasonal = decomp.seasonal.values
    residual = decomp.resid.values
    
        
    trend = trend.reshape((len(trend), 1))
    seasonal = seasonal.reshape((len(seasonal), 1))
    residual = residual.reshape((len(residual), 1))
     
    seasonal = seasonal.astype(int)    
    residual = residual.astype(int)
    trend = trend.astype(int)
    print(len(trend))
    trend_data.append(trend)
    seasonal_data.append(seasonal)
    residual_data.append(residual)

-->   1
2952
-->   2
2952
-->   3
2952
-->   4
2952
-->   5
2952
-->   6
2952
-->   7
2952
-->   8
2952
-->   9
2952
-->   10
2952
-->   11
2952
-->   12
2952
-->   13
2952
-->   14
2952
-->   15
2952
-->   16
2952
-->   17
2952
-->   18
2952
-->   19
2952
-->   20
2952
-->   21
2952
-->   22
2952
-->   23
2952
-->   24
2952
-->   25
2952
-->   26
2952
-->   27
2952
-->   28
2952
-->   29
2952
-->   30
2952
-->   31
2952
-->   32
2952
-->   33
2952
-->   34
2952
-->   35
2952
-->   36
2952
-->   37
2952
-->   38
2952
-->   39
2952
-->   40
2952
-->   41
2952
-->   42
2952
-->   43
2952
-->   44
2952
-->   45
2952
-->   46
2952
-->   47
2952
-->   48
2952
-->   49
2952
-->   50
2952
-->   51
2952
-->   52
2952
-->   53
2952
-->   54
2952
-->   55
2952
-->   56
2952
-->   57
2952
-->   58
2952
-->   59
2952
-->   60
2952
-->   61
2952
-->   62
2952
-->   63
2952
-->   64
2952
-->   65
2952
-->   66
2952
-->   67
2952
-->   68
2952
-->   69
2952
-->   70
2952
-->   71
2952
-->   72
2952
-

2952
-->   555
2952
-->   556
2952
-->   557
2952
-->   558
2952
-->   559
2952
-->   560
2952
-->   561
2952
-->   562
2952
-->   563
2952
-->   564
2952
-->   565
2952
-->   566
2952
-->   567
2952
-->   568
2952
-->   569
2952
-->   570
2952
-->   571
2952
-->   572
2952
-->   573
2952
-->   574
2952
-->   575
2952
-->   576
2952
-->   577
2952
-->   578
2952
-->   579
2952
-->   580
2952
-->   581
2952
-->   582
2952
-->   583
2952
-->   584
2952
-->   585
2952
-->   586
2952
-->   587
2952
-->   588
2952
-->   589
2952
-->   590
2952
-->   591
2952
-->   592
2952
-->   593
2952
-->   594
2952
-->   595
2952
-->   596
2952
-->   597
2952
-->   598
2952
-->   599
2952
-->   600
2952
-->   601
2952
-->   602
2952
-->   603
2952
-->   604
2952
-->   605
2952
-->   606
2952
-->   607
2952
-->   608
2952
-->   609
2952
-->   610
2952
-->   611
2952
-->   612
2952
-->   613
2952
-->   614
2952
-->   615
2952
-->   616
2952
-->   617
2952
-->   618
2952
-->   619
2952
-->   620
2952
-->  

KeyboardInterrupt: 

In [ ]:
final_data_trend = np.hstack((trend_data))
final_data_seasonal = np.hstack((seasonal_data))
final_data_residual = np.hstack((residual_data))

In [ ]:
final_data_residual[4]

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
      # find the end of this pattern
      end_ix = i + n_steps_in
      out_end_ix = end_ix + n_steps_out
      # check if we are beyond the dataset
      if out_end_ix > len(sequences):
          break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
      X.append(seq_x)
      y.append(seq_y)
  return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 24, 12

In [ ]:
# convert into input/output
X_seasonal, y_seasonal = split_sequences(final_data_seasonal, n_steps_in, n_steps_out)
X_trend, y_trend = split_sequences(final_data_trend, n_steps_in, n_steps_out)
X_residual, y_residual = split_sequences(final_data_residual, n_steps_in, n_steps_out)

count = 0
for i in range(len(X_seasonal)):
    count += 1
    print('inizia iterazione ', i)
    print(X_seasonal[i], y_seasonal[i])
    if count > 4: break

# flatten input season
n_input_seasonal = X_seasonal.shape[1] * X_seasonal.shape[2]
X_seasonal = X_seasonal.reshape((X_seasonal.shape[0], n_input_seasonal))
# flatten output
n_output_seasonal = y_seasonal.shape[1] * y_seasonal.shape[2]
y_seasonal = y_seasonal.reshape((y_seasonal.shape[0], n_output_seasonal))


# flatten input trend
n_input_trend = X_trend.shape[1] * X_trend.shape[2]
X_trend = X_trend.reshape((X_trend.shape[0], n_input_trend))
# flatten output
n_output_trend = y_trend.shape[1] * y_trend.shape[2]
y_trend = y_trend.reshape((y_trend.shape[0], n_output_trend))

# flatten input residual
n_input_residual = X_residual.shape[1] * X_residual.shape[2]
X_residual = X_residual.reshape((X_residual.shape[0], n_input_residual))
# flatten output residual
n_output_residual = y_residual.shape[1] * y_residual.shape[2]
y_residual = y_residual.reshape((y_residual.shape[0], n_output_residual))

In [ ]:
split_train_test = int(len(X_trend)*0.75)
train_X_trend, test_X_trend = X_trend[:split_train_test], X_trend[split_train_test:]
train_y_trend, test_y_trend = y_trend[:split_train_test], y_trend[split_train_test:]
print(len(train_X_trend), len(train_y_trend), len(test_X_trend), len(test_y_trend))

train_X_seasonal, test_X_seasonal = X_seasonal[:split_train_test], X_seasonal[split_train_test:]
train_y_seasonal, test_y_seasonal = y_seasonal[:split_train_test], y_seasonal[split_train_test:]
print(len(train_X_seasonal), len(train_y_seasonal), len(test_X_seasonal), len(test_y_seasonal))

train_X_residual, test_X_residual = X_residual[:split_train_test], X_residual[split_train_test:]
train_y_residual, test_y_residual = y_residual[:split_train_test], y_residual[split_train_test:]
print(len(train_X_residual), len(train_y_residual), len(test_X_residual), len(test_y_residual))

In [ ]:
# define model
model_trend = Sequential()
model_trend.add(Dense(500, activation='relu', input_dim=n_input_trend)) 
model_trend.add(Dense(250, activation='relu', input_dim=n_input_trend)) 

model_trend.add(Dense(n_output_trend))
model_trend.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_trend.fit(train_X_trend, train_y_trend, epochs=100, verbose=2)

In [ ]:
# define model
model_seasonal = Sequential()
model_seasonal.add(Dense(500, activation='relu', input_dim=n_input_seasonal)) 
model_seasonal.add(Dense(250, activation='relu', input_dim=n_input_seasonal))

model_seasonal.add(Dense(n_output_seasonal))
model_seasonal.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_seasonal.fit(train_X_seasonal, train_y_seasonal, epochs=100, verbose=2)

In [ ]:
model_residual = Sequential()
model_residual.add(Dense(500, activation='relu', input_dim=n_input_residual)) 
model_residual.add(Dense(250, activation='relu', input_dim=n_input_residual))

model_residual.add(Dense(n_output_residual))
model_residual.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_residual.fit(train_X_residual, train_y_residual, epochs=100, verbose=2)

In [ ]:
predicted_trend = np.array([])
for i in range(len(test_X_trend)):
    x_input = test_X_trend[i].reshape((1, n_input_trend))
    yhat = model_trend.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_trend = np.append(predicted_trend, yhat[0])

In [ ]:
predicted_seasonal = np.array([])
for i in range(len(test_X_seasonal)):
    x_input = test_X_seasonal[i].reshape((1, n_input_seasonal))
    yhat = model_seasonal.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_seasonal = np.append(predicted_seasonal, yhat[0])
    
predicted_residual = np.array([])    
for i in range(len(test_X_residual)):
    x_input = test_X_residual[i].reshape((1, n_input_trend))
    yhat = model_residual.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_residual = np.append(predicted_residual, yhat[0])

In [ ]:
predicted_final = predicted_trend + predicted_seasonal + predicted_residual

In [ ]:
predicted_final.shape

In [ ]:
num_data = []
## prepare dataset
for i, k in agg_by_cell:
    cell_i = agg_by_cell.get_group(i)
    cell_i = cell_i.iloc[::4, :]
    # define input sequence
    series_i = cell_i['nr_people'].values
    series_i = series_i.reshape((len(series_i), 1))
    num_data.append(series_i)

In [ ]:
final_data = hstack((num_data))
final_data.shape

In [ ]:
print(len(final_data))
print(final_data.shape[0] * final_data.shape[1])

In [ ]:
# convert into input/output
X, y = split_sequences(final_data, n_steps_in, n_steps_out)

count = 0
for i in range(len(X)):
    count += 1
    print('inizia iterazione ', i)
    print(X[i], y[i])
    if count > 1: break
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [ ]:
print(len(y))
print(len(y[3]))

In [ ]:
split_train_test = int(len(X)*0.75)
print(split_train_test)
train_X, test_X = X[:split_train_test], X[split_train_test:]
train_y, test_y = y[:split_train_test], y[split_train_test:]
print(len(train_X), len(train_y), len(test_X), len(test_y))

In [ ]:
print(test_y.shape[0], test_y.shape[1])

In [ ]:
exp_y = test_y.reshape((test_y.shape[0] * test_y.shape[1])) 
len(exp_y)

In [130]:
difference = abs(predicted_final - exp_y)
print('Mean Absolute Error = ', np.mean(difference))
plt.figure(figsize = (14,2))
plt.boxplot(difference, vert = False)
plt.show()

ValueError: operands could not be broadcast together with shapes (14191200,) (11852280,) 

In [ ]:
MAPE = np.mean(abs(100 * (difference/exp_y)))
MAPE